# Convert Year 11 Reporting Data to Daymap

In [1]:
# Python3 Script to import CSV containing student reporting data,
# automatically remove useless data, then append & re-format other
# data into suitable format for import into Daymap.
# 
# Author: Glen McKie & James Dundon
# e: glen.mckie709@schools.sa.edu.au
# e: james.dundon341@schools.sa.edu.au
# Organisation: Modbury High School
# 
# Current Version: in progress
# Date: December 2017

#CSV data should be organised using the included CSV titled Yr11_daymap_preparationSubject.csv


In [212]:
# Import pandas library
import pandas as pd

# Numpy libraryu not required at this stage...
# import numpy as np

In [226]:
# Read CSV file (Reporting) & input into a Pandas 'data frame'

# Define import path as variable "filename"
# This could be an input variable in later versions...

Input_CSV_File = 'Yr11_daymap_preparationSubject.csv'
print ('Attempting to import Reporting Data from file: "'+ Input_CSV_File + '"...')
Data_In = pd.read_csv(Input_CSV_File)



Attempting to import Reporting Data from file: "Yr11_daymap_preparationSubject.csv"...


In [241]:
# TRACER !!
# Check file import for correctness
print('Import of "' + Input_CSV_File + '" was successful!')
print(Data_In.tail())

Import of "Yr11_daymap_preparationSubject.csv" was successful!
      Student ID      Report Class Code        Item Code EnteredText
2215      170395  11 Physical Education  Subject grade 2           A
2216      170395  11 Physical Education  Subject grade 3           A
2217      170395     11 Scientific St B    Subject grade          C+
2218      170395     11 Scientific St B  Subject grade 2         NaN
2219      170395     11 Scientific St B  Subject grade 3           C


In [242]:
# Check for dodgy student data, may not catch all if have varied subject numbers.
grade_types = 3
Student_IDS = Data_In['Student ID'].astype('str')
uniqueIDS = []
for student in Student_IDS:
    if Student_IDS.str.count(student).sum() % grade_types != 0:
        if not student in uniqueIDS:
            uniqueIDS.append(student)

if len(uniqueIDS) > 0:
    for student in uniqueIDS:
        print((student)+':','does not have the correct number of grade attributes')
else:          
    print('All students have the correct number of grade attributes')

All students have the correct number of grade attributes


In [244]:
# Creates a list of student ID's
studentIDS = []
for i in range(len(Data_In.index)):
    if not Data_In.loc[i]['Student ID'] in studentIDS:
        studentIDS.append(Data_In.loc[i]['Student ID'])
print(len(studentIDS))

[130257, 130258, 140005, 140006, 140009, 140010, 140011, 140012, 140013, 140014, 140016, 140017, 140018, 140019, 140020, 140022, 140023, 140024, 140025, 140027, 140028, 140030, 140032, 140033, 140035, 140036, 140037, 140039, 140040, 140041, 140046, 140047, 140048, 140049, 140051, 140052, 140053, 140055, 140056, 140059, 140064, 140067, 140073, 140074, 140076, 140080, 140081, 140082, 140084, 140085, 140086, 140087, 140089, 140090, 140091, 140092, 140094, 140096, 140097, 140098, 140099, 140100, 140102, 140103, 140104, 140105, 140106, 140107, 140111, 140112, 140113, 140115, 140117, 140118, 140123, 140125, 140126, 140127, 140129, 140130, 140132, 140133, 140134, 140135, 140137, 140138, 140140, 140142, 140144, 140146, 140147, 140149, 140150, 140151, 140153, 140154, 140155, 140156, 140157, 140158, 140159, 140160, 140161, 140162, 140163, 140166, 140167, 140168, 140169, 140170, 140171, 140172, 140173, 140175, 140179, 140187, 140209, 140222, 140252, 150256, 150270, 150298, 150322, 150324, 160088,

In [231]:
#Check the length of Index
print('Length of index:',len(Data_In.index))

Length of index: 2220


In [248]:
# Required CSV format is:
# Column 1 = Student_Code
# Columns 2:_ = Subjects

# Number of subjects
Num_subjects = 7

# Set up a data frame to hold data in converted format

print('Setting up new data frame to hold imported data...')

#Required_Format = {'StudentCode' : [''],'Subject1' : [''], 'Subject2' : [''] ,'Subject3' : [''] ,'Subject4' : [''],'Subject5' : [''],'Subject6' : [''],'Subject7' : ['']}
Required_Format = {'StudentCode' : ['']}
for i in range(Num_subjects):
    Required_Format['Subject'+str(i+1)] = ['']



Data_Out = pd.DataFrame(Required_Format)

# Specify order of Data_Out columns

#Data_Out = Data_Out[['StudentCode','Subject1','Subject2','Subject3','Subject4','Subject5','Subject6','Subject7']]

# Would like to automate this based on Num_subjects
row = pd.Series(['','','','','','','',''], index = ['StudentCode','Subject1','Subject2','Subject3','Subject4','Subject5','Subject6','Subject7'])

# Creates an empty data frame with rows equalling the length of unique student ID's
for i in range(len(studentIDS)-1):
                Data_Out = Data_Out.append(row,ignore_index=True)

#Check structure
print(Data_Out.tail())

Setting up new data frame to hold imported data...
    StudentCode Subject1 Subject2 Subject3 Subject4 Subject5 Subject6 Subject7
124                                                                           
125                                                                           
126                                                                           
127                                                                           
128                                                                           


In [249]:
# Create variable to act as index within Data_In AND Data_Out df...
data_out_index = 0
data_in_index = 0

# Grade types ie term, exam, semester
grade_types = 3

# Loop through each student ID
for ID in studentIDS:

    #While the student ID matches a row in the dataframe with that ID
    while ID == Data_In.loc[data_in_index]['Student ID']:
        # Extract Student ID from CSV & place data into df 'Data_Out' as Student Code

        Data_Out.loc[data_out_index]['StudentCode'] = Data_In.loc[data_in_index]['Student ID']
        student_code = Data_In.loc[data_in_index]['Student ID']

        # Required data for each subject is "SubjectName: Term 2, Grade, Exam, Grade, Semester, Grade" 
        # Term_Grade = Subject grade in INPUT file
        # Exam_Grade = Subject grade2 in INPUT file
        # Semester_Grade = Subject grade3 in INPUT file

        ############################## 

        # Extract SUBJECT data & place data into df 'Data_Out' in required format
        # if student_code in row matches current student (handles limited subjects for a student)
        # Cycles through for number of subjects in Num_subjects
        for i in range(Num_subjects):
            if  student_code == Data_In.loc[data_in_index]['Student ID']:
                subject_name = str(Data_In.loc[data_in_index]['Report Class Code'])
                term_grade = str(Data_In.loc[data_in_index]['EnteredText'])
                exam_grade = str(Data_In.loc[data_in_index+1]['EnteredText'])
                semester_grade = str(Data_In.loc[data_in_index+2]['EnteredText'])
                # Deal with nan grade bug
                if(term_grade) == 'nan':
                        term_grade = str('NA')
                if(exam_grade) == 'nan':
                        exam_grade = str('NA')
                if(semester_grade) == 'nan':
                        semester_grade = str('NA')

                Data_Out.loc[data_out_index]['Subject'+str(i+1)] = subject_name + ": " + "Term 4, " + term_grade + ", Exam, " + exam_grade +", Semester, " + semester_grade

                # Output converted SUBJECT data for checking
                converted_summary = Data_Out.loc[data_out_index,'Subject'+str(i+1)]
                # To print output, uncomment line below
                #print(converted_summary)
                # Move to next subject within the same student's data in Data_In
                if data_in_index + 3 > len(Data_In.index)-1:
                    break
                else:
                    data_in_index = data_in_index + grade_types
        
        data_out_index += 1
        if data_in_index + grade_types > len(Data_In.index)-1:
            break

#All data loop Finished    
print('New Data Frame Complete!')

New Data Frame Complete!


In [250]:
# Output resulting df 'Data_Out' to CSV
# In future the path could be input by user or similar -
# currently the filename is static.

# NB : this works, so do not touch!

Data_Out.to_csv('Output_'+Input_CSV_File, encoding='utf-8', index=False)

In [251]:
print ('CSV has been output... time for a beer!')

CSV has been output... time for a beer!


In [253]:
print(Data_Out.tail())

,StudentCode,Subject1,Subject2,Subject3,Subject4,Subject5,Subject6,Subject7
124,160088,"11 English: Term 4, B, Exam, NA, Semester, B","11 Info Pro & Pub: Term 4, D, Exam, NA, Semest...","11 Legal Studies: Term 4, C, Exam, C, Semester, C","11 Maths General: Term 4, B, Exam, B, Semester, B","11 Outdoor Const: Term 4, A, Exam, NA, Semeste...","11 Sport & Recreation: Term 4, B, Exam, NA, Se...",
125,160245,"11 English: Term 4, B, Exam, NA, Semester, B","11 History: Term 4, C, Exam, C+, Semester, C","11 Hospitality: Term 4, B, Exam, NA, Semester, A","11 Legal Studies: Term 4, E, Exam, D, Semester, D","11 Maths General: Term 4, blank, Exam, blank, ...","11 Outdoor Const: Term 4, A, Exam, NA, Semeste...",
126,170353,"11 ESL: Term 4, C+, Exam, A, Semester, C","11 Maths General: Term 4, D, Exam, C, Semester, D","11 Metalwork: Term 4, B, Exam, NA, Semester, B","11 Physical Education: Term 4, C, Exam, C, Sem...","11 Research Practices: Term 4, C-, Exam, NA, S...","11 Woodwork: Term 4, B, Exam, NA, Semester, B",
127,170359,"11 Chemistry: Term 4, A, Exam, A, Semester, A","11 English Studies: Term 4, A, Exam, A, Semest...","11 Maths Studies 3: Term 4, A, Exam, A, Semest...","11 Maths Studies 4: Term 4, A, Exam, A, Semest...","11 Physics: Term 4, A, Exam, B, Semester, A",,
128,170395,"10 PLP: Term 4, C-, Exam, NA, Semester, C-","11 Biology: Term 4, B, Exam, A, Semester, B","11 English: Term 4, B, Exam, NA, Semester, B","11 Maths General: Term 4, C, Exam, A, Semester, B","11 Nutrition: Term 4, C, Exam, NA, Semester, C","11 Physical Education: Term 4, A, Exam, A, Sem...","11 Scientific St B: Term 4, C+, Exam, NA, Seme..."
